In [ ]:
# Runtime is about 1 hour

In [ ]:
import numpy as np
import os
import torch
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch import nn
from torch.utils.data import DataLoader, TensorDataset, Dataset, Subset
import torch.nn.utils.prune as prune
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torchvision import transforms as T
from torchvision.transforms import ToTensor 
from utils.train import train_model,set_dataset

In [ ]:
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

In [ ]:
target_model = torch.load("./models/victims/Cifar10_Resnet18.pt",map_location='cpu')
target_model = target_model.to(device)

In [ ]:
#setting train hyper-parameter
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(target_model.parameters(), lr=0.00005, momentum=0.9, weight_decay=5e-4)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [ ]:
train_data, test_data, class_num = set_dataset("cifar10")

In [ ]:
batch_size = 128
train_loader = torch.utils.data.DataLoader(
                 dataset=train_data,
                 batch_size=batch_size,
                 shuffle=True)
test_loader = torch.utils.data.DataLoader(
                dataset=test_data,
                batch_size=batch_size,
                shuffle=True)

In [ ]:
if not os.path.exists("./models/attack_model/MPA"):
    os.makedirs("./models/attack_model/MPA")
for amount in [0.2, 0.4, 0.6]:
    for name, module in target_model.named_modules():
        if isinstance(module, torch.nn.Conv2d):
            prune.l1_unstructured(module, name='weight', amount=amount)
        elif isinstance(module, torch.nn.Linear):
            prune.l1_unstructured(module, name='weight', amount=amount)
        elif isinstance(module, torch.nn.BatchNorm2d):
            prune.l1_unstructured(module, name='weight', amount=amount)
    model, acc = train_model(target_model, criterion, optimizer_ft, exp_lr_scheduler,device, train_loader, test_loader, 5)
    torch.save(model,'./models/attack_model/MPA/Prune_'+str(int(amount*100))+'.pt')
    acc_file=open('./models/attack_model/MPA/Prune_'+str(int(amount*100))+'.txt','w')
    acc_file.write(str(acc))
    acc_file.close()

In [ ]:
#MNIST version
target_model = torch.load("./models/test/MNIST_Resnet18.pt",map_location='cpu')
target_model = target_model.to(device)

In [ ]:
#setting train hyper-parameter
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(target_model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [ ]:
train_data, test_data, class_num = set_dataset("MNIST")

In [ ]:
batch_size = 128
train_loader = torch.utils.data.DataLoader(
                 dataset=train_data,
                 batch_size=batch_size,
                 shuffle=True)
test_loader = torch.utils.data.DataLoader(
                dataset=test_data,
                batch_size=batch_size,
                shuffle=True)

In [ ]:
for amount in [0.2, 0.4, 0.6]:
    target_model = torch.load("./models/test/MNIST_Resnet18.pt",map_location='cpu')
    target_model = target_model.to(device)
    for name, module in target_model.named_modules():
        if isinstance(module, torch.nn.Conv2d):
            prune.l1_unstructured(module, name='weight', amount=amount)
        elif isinstance(module, torch.nn.Linear):
            prune.l1_unstructured(module, name='weight', amount=amount)
        elif isinstance(module, torch.nn.BatchNorm2d):
            prune.l1_unstructured(module, name='weight', amount=amount)
    model, acc = train_model(target_model, criterion, optimizer_ft, exp_lr_scheduler,device, train_loader, test_loader, 5)
    torch.save(model,'./models/attack_model/MPA/MNIST_Prune_'+str(int(amount*100))+'.pt')
    acc_file=open('./models/attack_model/MPA/MNIST_Prune_'+str(int(amount*100))+'.txt','w')
    acc_file.write(str(acc))
    acc_file.close()